##  Inferencing on Mistral 7B LLM with 4-bit quantization 🚀 - In FREE Google Colab

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=eovBbABk3hw&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)

[![Imgur](https://imgur.com/Lz4ov4K.png)](https://www.youtube.com/watch?v=eovBbABk3hw&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)



In [1]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'bn22/Mistral-7B-Instruct-v0.1-sharded'

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]

text = "[INST] How AI will replace Engineers [/INST]"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [13]:
text = """
O texto a seguir é uma transcrição de atendimento:

Atendente: Obrigado por ligar para o Serviço de Atendimento ao Cliente da ABC Eletrônicos. Meu nome é Maria. Como posso ajudar você hoje?
Cliente: Olá, Maria. Meu nome é João, e tenho enfrentado muitos problemas com o produto que comprei da sua empresa. Comprei um smartphone modelo ABC123 há algumas semanas, e ele só tem me dado problemas.
Atendente: Lamento ouvir isso, João. Entendo como isso pode ser frustrante. Você poderia descrever os problemas que está enfrentando com o seu smartphone ABC123?
Cliente: Bem, a vida útil da bateria é terrível, e o aparelho reinicia aleatoriamente. A qualidade da câmera não corresponde ao que foi anunciado, e ele trava às vezes. Estou realmente decepcionado com essa compra.
Atendente: Peço desculpas pela inconveniência que você está enfrentando. Levamos essas questões a sério, e gostaria de ajudar a resolver esses problemas. Vamos começar verificando algumas coisas. Você tentou reiniciar o telefone ou realizar uma restauração de fábrica?
Cliente: Sim, tentei essas etapas, e os problemas persistem. Atualizei o software também, mas não resolveu.
Atendente: Obrigada por tentar essas etapas, João. Como o seu telefone ainda está na garantia, podemos enviar a você um aparelho de substituição ou agendar um técnico para avaliar e reparar o seu telefone atual. Qual opção você prefere?
Cliente: Acho que gostaria de optar pela substituição, Maria. Tive muitos problemas com esse telefone, e preciso de um aparelho confiável.
Atendente: Entendi, João. Vou processar o pedido de substituição para você. Deve chegar nos próximos dias úteis. Também vou enviar a você os detalhes de envio e as instruções para devolver o seu telefone atual por e-mail.
Cliente: Obrigado, Maria. Agradeço sua ajuda na resolução deste problema. Espero que o aparelho de substituição funcione melhor.
Atendente: De nada, João. Estamos aqui para garantir a satisfação dos nossos clientes. Se você tiver algum problema com a substituição ou tiver outras preocupações, não hesite em nos contatar. Há mais alguma coisa em que posso ajudar hoje?
Cliente: Não, é só isso por enquanto. Obrigado pela sua assistência, Maria.
Atendente: Foi um prazer, João. Tenha um ótimo dia, e esperamos que o novo telefone atenda às suas expectativas.

Identifique as reclamações do cliente.
"""

In [ ]:
encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded
generated_ids = model.generate(**model_input, max_new_tokens=400, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])